In [1]:
import pandas as pd

In [2]:
import re
import string

In [3]:
from nltk.corpus import stopwords
import nltk

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
from sklearn.decomposition import NMF

In [126]:
df_union = pd.read_csv('df_union.csv')  

In [7]:
df_union

,author_id,author_location,author_num_cities,author_num_helpful_votes,author_num_reviews,author_num_type_reviews,author_username,date,date_stayed,id,...,address_postal-code,address_region,address_street-address,details,hotel_class,name,phone,region_id,type,url
0,2E4442B5FF4C485F4EF4FD49B66AD171,"Raleigh, NC",10.0,32.0,16.0,14.0,shaniz1,"July 11, 2008",July 2008,17784086.0,...,19107,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...
1,556A4A70DCA457DE95FD95B9CE217014,"Nashville, TN",5.0,2.0,12.0,5.0,Grumpylicous,"August 4, 2012",September 2011,136233716.0,...,19107,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...
2,8D6ECB2EB034591EF86E540319D24992,New York,13.0,55.0,21.0,20.0,AA-Platinum,"November 21, 2006",November 2006,6161247.0,...,19107,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...
3,DDCEA80A1BFE32D8C91EFE473737A229,New York,34.0,74.0,50.0,14.0,DavidTrip,"August 12, 2009",August 2009,37482297.0,...,19107,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...
4,19568376B8C1295886B69B322E4C6055,"Harrisburg, Pennsylvania",10.0,3.0,11.0,7.0,unclesamspad2,"May 30, 2009",May 2009,31138794.0,...,19107,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,61B3197983D553C6CCC12D3C265E4375,Mexico,19.0,18.0,30.0,28.0,AlejandraMexico,"May 10, 2009",May 2009,29721005.0,...,78258,TX,125 North Loop 1604 West,NaN,3.0,Homewood Suites by Hilton Hotel San Antonio North,NaN,60956.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60956...
9996,9CE843F2646875FCB4EE774CAB6FCB68,"Detroit, Michigan",31.0,25.0,55.0,22.0,mjwingfans,"July 3, 2010",May 2010,69543852.0,...,46250,IN,6817 E. 82nd Street,NaN,2.5,Hampton Inn Indianapolis - NE / Castleton,NaN,37209.0,hotel,http://www.tripadvisor.com/Hotel_Review-g37209...
9997,7BFD165584BDE13CBD73A9AF1C1B2BF7,Austin,9.0,11.0,17.0,9.0,AustinBarb,"December 30, 2011",December 2011,122265474.0,...,78216,TX,850 Halm Boulevard,NaN,2.5,La Quinta Inn & Suites San Antonio Airport,NaN,60956.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60956...
9998,7B5194085DEE03D4FCC5589C6353D683,near Liège,5.0,6.0,5.0,5.0,elpelso,"March 5, 2008",March 2008,14063603.0,...,77003,TX,1810 Bell Ave,NaN,2.5,Holiday Inn Express Houston-Downtown Conventio...,NaN,56003.0,hotel,http://www.tripadvisor.com/Hotel_Review-g56003...


## Text preprocessing

In [9]:
df_union.columns

Index(['author_id', 'author_location', 'author_num_cities',
       'author_num_helpful_votes', 'author_num_reviews',
       'author_num_type_reviews', 'author_username', 'date', 'date_stayed',
       'id', 'num_helpful_votes', 'offering_id', 'ratings_check_in_front_desk',
       'ratings_cleanliness', 'ratings_location', 'ratings_overall',
       'ratings_rooms', 'ratings_service', 'ratings_value', 'text', 'title',
       'via_mobile', 'ratings_sleep_quality',
       'ratings_business_service_(e_g_internet_access)', 'address_locality',
       'address_postal-code', 'address_region', 'address_street-address',
       'details', 'hotel_class', 'name', 'phone', 'region_id', 'type', 'url'],
      dtype='object')

Delete reviews that contains more non-english words than english, because a lot of french words defined as english

In [67]:
words = set(nltk.corpus.words.words())

In [62]:
df_union.text[df_union.text == '0']

0       0
1       0
2       0
3       0
4       0
       ..
9994    0
9996    0
9997    0
9998    0
9999    0
Name: text, Length: 8790, dtype: object

In [50]:

words = set(nltk.corpus.words.words())

sent = "Io andiamo to the beach with my amico."
" ".join(w for w in nltk.wordpunct_tokenize(sent) \
         if w.lower() in words or not w.isalpha())
# 'Io to the beach with my'

'Io to the beach with my .'

In [127]:
# Compound Term Extraction
#repl = lambda x: x.replace('new york', 'new_york').replace('fashion week', 'fashion_week').replace('new year', 'new_year').replace('self care', 'self_care')

# Text preprocessing steps - remove numbers, captial letters and punctuation
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())

words = set(nltk.corpus.words.words())
english_only = lambda x: " ".join(w for w in nltk.wordpunct_tokenize(x) \
         if w.lower() in words or not w.isalpha())

df_union['text'] = df_union.text.map(alphanumeric).map(punc_lower)#.map(english_only) #.map(repl)
df_union['title'] = df_union.title.map(alphanumeric).map(punc_lower)#.map(english_only) #.map(repl)
df_union.head()

,author_id,author_location,author_num_cities,author_num_helpful_votes,author_num_reviews,author_num_type_reviews,author_username,date,date_stayed,id,...,address_postal-code,address_region,address_street-address,details,hotel_class,name,phone,region_id,type,url
0,2E4442B5FF4C485F4EF4FD49B66AD171,"Raleigh, NC",10.0,32.0,16.0,14.0,shaniz1,"July 11, 2008",July 2008,17784086.0,...,19107,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...
1,556A4A70DCA457DE95FD95B9CE217014,"Nashville, TN",5.0,2.0,12.0,5.0,Grumpylicous,"August 4, 2012",September 2011,136233716.0,...,19107,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...
2,8D6ECB2EB034591EF86E540319D24992,New York,13.0,55.0,21.0,20.0,AA-Platinum,"November 21, 2006",November 2006,6161247.0,...,19107,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...
3,DDCEA80A1BFE32D8C91EFE473737A229,New York,34.0,74.0,50.0,14.0,DavidTrip,"August 12, 2009",August 2009,37482297.0,...,19107,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...
4,19568376B8C1295886B69B322E4C6055,"Harrisburg, Pennsylvania",10.0,3.0,11.0,7.0,unclesamspad2,"May 30, 2009",May 2009,31138794.0,...,19107,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...


In [111]:
df_union.iloc[0]['date']

'July 11, 2008'

In [130]:
df_union['Is_English'] = 1

In [133]:
# check if review written in English
# all review that contain at least a half of English words mark as english, otherway - not

for i, row in enumerate(df_union.text):
    len_review = len(row)
    english_review = " ".join(w for w in nltk.wordpunct_tokenize(row) \
         if w.lower() in words or not w.isalpha())
    
    len_english = len(english_review)
    
    if len_review > len_english * 2:
        df_union.Is_English[i] = 0
    else:
        df_union.Is_English[i] = 1

<ipython-input-133-f237dad4a5c0>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_union.Is_English[i] = 1
<ipython-input-133-f237dad4a5c0>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_union.Is_English[i] = 0


In [135]:
df_union.text[df_union.Is_English == 0]

9       middenin downtown la een hotel van grote klass...
13      positivo \ngrandioso hall con unos techos espe...
16      hotel extrement bien situé idéal pour pouvoir ...
17      dopo aver fatto una ricerca approfondita degli...
20      ci siamo trovati benissimo  camere confortevol...
                              ...                        
9936    j ai voulu essayé cet hôtel   étoiles pour une...
9940    das motel liegt verkehrsgünstig um die stadt p...
9952    comme son nom l indique  l hôtel est en plein ...
9987    hotellet ligger sentralt i san francisco  ikke...
9995    definitivamente recomendaría este hotel   los ...
Name: text, Length: 1231, dtype: object

In [136]:
df_english = df_union[(df_union['Is_English'] == 1)]

In [137]:
df_english 

,author_id,author_location,author_num_cities,author_num_helpful_votes,author_num_reviews,author_num_type_reviews,author_username,date,date_stayed,id,...,address_region,address_street-address,details,hotel_class,name,phone,region_id,type,url,Is_English
0,2E4442B5FF4C485F4EF4FD49B66AD171,"Raleigh, NC",10.0,32.0,16.0,14.0,shaniz1,"July 11, 2008",July 2008,17784086.0,...,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...,1
1,556A4A70DCA457DE95FD95B9CE217014,"Nashville, TN",5.0,2.0,12.0,5.0,Grumpylicous,"August 4, 2012",September 2011,136233716.0,...,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...,1
2,8D6ECB2EB034591EF86E540319D24992,New York,13.0,55.0,21.0,20.0,AA-Platinum,"November 21, 2006",November 2006,6161247.0,...,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...,1
3,DDCEA80A1BFE32D8C91EFE473737A229,New York,34.0,74.0,50.0,14.0,DavidTrip,"August 12, 2009",August 2009,37482297.0,...,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...,1
4,19568376B8C1295886B69B322E4C6055,"Harrisburg, Pennsylvania",10.0,3.0,11.0,7.0,unclesamspad2,"May 30, 2009",May 2009,31138794.0,...,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,611A2AACA2267A12D25AFF59523DBC04,Columbia Maryland,10.0,26.0,11.0,10.0,LoveLife82,"October 29, 2008",October 2008,21389392.0,...,TX,2838 Cinema Ridge,NaN,3.0,Residence Inn San Antonio Seaworld,NaN,60956.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60956...,1
9996,9CE843F2646875FCB4EE774CAB6FCB68,"Detroit, Michigan",31.0,25.0,55.0,22.0,mjwingfans,"July 3, 2010",May 2010,69543852.0,...,IN,6817 E. 82nd Street,NaN,2.5,Hampton Inn Indianapolis - NE / Castleton,NaN,37209.0,hotel,http://www.tripadvisor.com/Hotel_Review-g37209...,1
9997,7BFD165584BDE13CBD73A9AF1C1B2BF7,Austin,9.0,11.0,17.0,9.0,AustinBarb,"December 30, 2011",December 2011,122265474.0,...,TX,850 Halm Boulevard,NaN,2.5,La Quinta Inn & Suites San Antonio Airport,NaN,60956.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60956...,1
9998,7B5194085DEE03D4FCC5589C6353D683,near Liège,5.0,6.0,5.0,5.0,elpelso,"March 5, 2008",March 2008,14063603.0,...,TX,1810 Bell Ave,NaN,2.5,Holiday Inn Express Houston-Downtown Conventio...,NaN,56003.0,hotel,http://www.tripadvisor.com/Hotel_Review-g56003...,1


In [13]:
# removing stop words from the text
stop = stopwords.words('english')
stop += ['.', ',', '(', ')', "'", '"']
#stop += ['http', 'https', 'co', 'see', 'editor', 'writer', 'vogue', 'best', 'look', 'one', 'may', 'make']
stop = set(stop)

## Document-Term Matrix

In [138]:
# The first document-term matrix has default Count Vectorizer values - counts of unigrams

vectorizer = CountVectorizer(stop_words = stop)

doc_word = vectorizer.fit_transform(df_english.text)

pd.DataFrame(doc_word.toarray(), columns=vectorizer.get_feature_names()).head()

,aa,aaa,aaaaaaaaaaaaaaaaaaaaaaaaaa,aaaawwwwesome,aapointed,aargh,aaron,aarp,aback,abandoned,...,zombies,zone,zones,zoning,zoo,zoomed,zooming,zoribel,zorro,zuli
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Topic modeling

### Non-Negative Matrix Factorization (NMF)

In [139]:
nmf_model = NMF(10)
doc_topic_nmf = nmf_model.fit_transform(doc_word)
doc_topic_nmf.shape

/Users/albinasitdikova/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1076: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


(8769, 10)

In [140]:
topic_word_nmf = nmf_model.components_
topic_word_nmf.shape

(10, 22494)

In [141]:
words_nmf = vectorizer.get_feature_names()
t = nmf_model.components_.argsort(axis=1)[:,-1:-11:-1]
topic_words_nmf = [[words_nmf[e] for e in l] for l in t]
topic_words_nmf

[['hotel',
  'city',
  'new',
  'located',
  'times',
  'street',
  'lobby',
  'parking',
  'right',
  'recommend'],
 ['room',
  'bed',
  'floor',
  'bathroom',
  'small',
  'view',
  'king',
  'could',
  'tv',
  'night'],
 ['great',
  'location',
  'service',
  'time',
  'place',
  'really',
  'view',
  'get',
  'city',
  'food'],
 ['us',
  'back',
  'day',
  'went',
  'check',
  'could',
  'told',
  'wanted',
  'made',
  'breakfast'],
 ['good',
  'breakfast',
  'free',
  'area',
  'location',
  'also',
  'walk',
  'one',
  'clean',
  'well'],
 ['stay',
  'would',
  'night',
  'place',
  'definitely',
  'clean',
  'recommend',
  'really',
  'stayed',
  'comfortable'],
 ['staff',
  'friendly',
  'helpful',
  'clean',
  'well',
  'stayed',
  'comfortable',
  'desk',
  'front',
  'service'],
 ['rooms',
  'two',
  'small',
  'beds',
  'clean',
  'location',
  'floor',
  'new',
  'stayed',
  'noise'],
 ['nice',
  'really',
  'area',
  'also',
  'bathroom',
  'pool',
  'lobby',
  'like',
  

In [54]:
from nltk.corpus import words

In [55]:
"che" in words.words()

True

### Try using TF-IDF instead of Count Vectorizer

In [142]:
# Create TF-IDF versions of the Count Vectorizers created earlier in the exercise
from sklearn.feature_extraction.text import TfidfVectorizer

In [143]:
tfidf = TfidfVectorizer(stop_words=stop)

In [144]:
doc_word_tfidf = tfidf.fit_transform(df_union.text)              

In [145]:
pd.DataFrame(doc_word_tfidf.toarray(), columns=tfidf.get_feature_names()).head()

,aa,aaa,aaaaaaaaaaaaaaaaaaaaaaaaaa,aaaawwwwesome,aan,aandringen,aangeboden,aangenaam,aangeraden,aankomst,...,駅も近くて交通の便もいいです,駐車場があるのと,駐車場のスタッフに提示する必要があります,駐車場はありませんが,駐車場は地下にありますが,高層階は川景色も美しく,ｈｉｌｔｏｎ,ｌｇ社製の大型液晶テレビがありました,ｎｙっぽい,ｔｖに表示される時刻がそれぞれ違っており
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### NMF - tf-ifd

In [146]:
nmf_model_2 = NMF(10)
doc_topic_nmf_2 = nmf_model_2.fit_transform(doc_word_tfidf)
doc_topic_nmf_2.shape

(10000, 10)

In [147]:
topic_word_nmf_2 = nmf_model_2.components_
topic_word_nmf_2.shape

(10, 38937)

In [148]:
words_2 = tfidf.get_feature_names()
t_2 = nmf_model_2.components_.argsort(axis=1)[:,-1:-11:-1]
topic_words_nmf_2 = [[words_2[e] for e in l] for l in t_2]
topic_words_nmf_2

[['good',
  'nice',
  'hotel',
  'breakfast',
  'free',
  'area',
  'rooms',
  'parking',
  'pool',
  'downtown'],
 ['de', 'et', 'est', 'le', 'très', 'la', 'pas', 'pour', 'les', 'un'],
 ['und', 'das', 'die', 'ist', 'der', 'sehr', 'zimmer', 'war', 'zu', 'im'],
 ['staff',
  'friendly',
  'stay',
  'helpful',
  'clean',
  'comfortable',
  'would',
  'hotel',
  'definitely',
  'rooms'],
 ['de', 'el', 'que', 'en', 'la', 'muy', 'es', 'las', 'con', 'para'],
 ['us', 'hotel', 'desk', 'front', 'time', 'one', 'get', 'back', 'stay', 'day'],
 ['di', 'la', 'il', 'per', 'che', 'non', 'molto', 'un', 'con', 'una'],
 ['hotel',
  'square',
  'times',
  'new',
  'york',
  'nyc',
  'central',
  'subway',
  'city',
  'walk'],
 ['room',
  'bed',
  'bathroom',
  'floor',
  'small',
  'night',
  'shower',
  'view',
  'tv',
  'two'],
 ['great',
  'location',
  'service',
  'place',
  'bar',
  'really',
  'view',
  'food',
  'chicago',
  'loved']]

In [149]:
doc_topic_nmf_2

array([[4.20338813e-03, 6.21452904e-05, 0.00000000e+00, ...,
        1.05730252e-02, 3.32004933e-02, 1.10223290e-03],
       [5.10088904e-02, 0.00000000e+00, 0.00000000e+00, ...,
        2.60285950e-02, 2.03333305e-02, 2.94835705e-03],
       [5.96609388e-03, 0.00000000e+00, 9.25592265e-04, ...,
        0.00000000e+00, 2.31347841e-02, 0.00000000e+00],
       ...,
       [1.93412461e-02, 0.00000000e+00, 1.40136517e-04, ...,
        0.00000000e+00, 3.78987776e-02, 2.44743075e-02],
       [3.79840261e-02, 0.00000000e+00, 0.00000000e+00, ...,
        1.79469549e-02, 3.08444813e-03, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.11633215e-02, 6.60726276e-03, 0.00000000e+00]])

In [150]:
H = pd.DataFrame(doc_topic_nmf_2.round(5),
                 index = df_union.text
                )
H

,0,1,2,3,4,5,6,7,8,9
text,,,,,,,,,,
i recently stayed here for two nights while in town for a conference check in was smooth and took about minutes i was surprised to learn that the clerk gave me a room with a view of city hall well what that meant was you have a room that borders a very busy street i felt like i was trying to sleep at a nascar event if you stay here make sure you get a room that faces away from city hall \ni received my receipt under the door in the morning and it had a food charge for which was not mine i brought it to the attention of the checkout desk and i was told that they would need to research it out with the restaurant manager i guess guilty until proven innocent they have subsequently removed it from my bill i still do not appreciate the inconvenience \nthe hotel over all is beautiful and the rooms are spacious,0.00420,0.00006,0.00000,0.01046,0.00000,0.04524,0.00080,0.01057,0.03320,0.00110
i stayed here for a business trip and stayed extra nights room was very clean had a nice view i will def stay here when i visit again the restaurant downstairs was good it was close to the convention center and other areas for walking the city i was able to walk to independence hall no problem,0.05101,0.00000,0.00000,0.03244,0.00000,0.00000,0.00000,0.02603,0.02033,0.00295
after a disappointing night at the hampton inn i moved my family here what a difference the room was spacious and comfortable the staff was suburb and super friendly \nthe only problem we had was with a chamber maid who refused to not enter the room eventhough the baby was sleeping management to their credit was very helpful and apologetic this is a place i would definitely recommend \nthis is one of theee marriott properties next to each other the others being the marriott downtown and the residence inn we were there for a conference at the marriott downtown but were able to stay at the courtyard for about less if you go out the back door the hotels are literally across the street from each other,0.00597,0.00000,0.00093,0.05327,0.00000,0.02045,0.00000,0.00000,0.02313,0.00000
we stayed at the marriott courtyard in downtown philadelphia for one night in august \nthe hotel is across the street from city hall and about a block from reading terminal market the hotel is also right next door to the marriott residence inn and right behind the large marriott the hotel has a landmark facade and lobby so it appears quite elegant the staff was very helpful and alert the rooms are nice but nothing super special old tvs marginal pillows \nthe indoor pool is nice for a quick cool down and the fitness center is very well equipped,0.05574,0.00000,0.00154,0.01966,0.00080,0.00285,0.00076,0.02163,0.01217,0.00000
the hotels exterior blends in well with the awe inspiring city hall right across the street the interior was recently don t know when upgraded and has that newer hotel feel our room was clean and comfortable and looked like a basic courtyard hotel room we ate sandwiches at the hotels restaraunt wich was good but not great the atmosphere was relaxing and inviting wich was a needed retreat after beeing on the busy streets of philly best tips don t use mapquest to find this hotel it s not correct use the valet parking it s a steep a day but finding good parking can be difficult you will have to walk alot or take the philly flash to get around,0.04246,0.00000,0.00047,0.00177,0.00016,0.01357,0.00000,0.01601,0.01287,0.00851
...,...,...,...,...,...,...,...,...,...,...
definitivamente recomendaría este hotel los cuartos son amplios limpios bien decorados comodos el desayuno gratis es bueno cereal leche cafe jugo fruta waffles huevo bagels mas que suficiente para empezar el día la ubicación mmmm esta un poco lejos de downtown a unos mins aproximadamente en coche \nmuy cerca del hotel hay un restaurante japones que se llama sushizushi buenisimo \nen general nuestra experiencia fue buena definitivamente regresaremos a homewood suites \nmi unica queja es con respecto a

### !!! try different topic modeling on tf-idf

### LSA

In [151]:
from sklearn.decomposition import TruncatedSVD

In [152]:
# Acronynms: Latent Semantic Analysis (LSA) is just another name for 
#  Signular Value Decomposition (SVD) applied to Natural Language Processing (NLP)
lsa = TruncatedSVD(10)
doc_topic_lsa = lsa.fit_transform(doc_word_tfidf)
lsa.explained_variance_ratio_

array([0.00712282, 0.0079588 , 0.00648157, 0.00515626, 0.00387008,
       0.00351218, 0.00317251, 0.00305936, 0.00292312, 0.00282576])

In [153]:
doc_topic_lsa

array([[ 0.20468594, -0.00741533, -0.00822931, ...,  0.00679025,
        -0.00594583,  0.00841064],
       [ 0.22379838, -0.00744985, -0.00637915, ...,  0.01796237,
        -0.01130806,  0.00740407],
       [ 0.17807719, -0.00987592, -0.00275508, ..., -0.02697782,
         0.01279491, -0.04537998],
       ...,
       [ 0.23864172, -0.01250693, -0.00534082, ..., -0.06092117,
        -0.00640697,  0.02437539],
       [ 0.15272531, -0.00295106,  0.00107963, ..., -0.0034223 ,
        -0.08744373, -0.00471721],
       [ 0.18036345, -0.00561332, -0.00362238, ...,  0.0069374 ,
        -0.04059883, -0.05797217]])

In [154]:
Vt = pd.DataFrame(doc_topic_lsa.round(5),
             index = df_union.text)
Vt

,0,1,2,3,4,5,6,7,8,9
text,,,,,,,,,,
i recently stayed here for two nights while in town for a conference check in was smooth and took about minutes i was surprised to learn that the clerk gave me a room with a view of city hall well what that meant was you have a room that borders a very busy street i felt like i was trying to sleep at a nascar event if you stay here make sure you get a room that faces away from city hall \ni received my receipt under the door in the morning and it had a food charge for which was not mine i brought it to the attention of the checkout desk and i was told that they would need to research it out with the restaurant manager i guess guilty until proven innocent they have subsequently removed it from my bill i still do not appreciate the inconvenience \nthe hotel over all is beautiful and the rooms are spacious,0.20469,-0.00742,-0.00823,-0.09898,-0.00445,0.03335,-0.00204,0.00679,-0.00595,0.00841
i stayed here for a business trip and stayed extra nights room was very clean had a nice view i will def stay here when i visit again the restaurant downstairs was good it was close to the convention center and other areas for walking the city i was able to walk to independence hall no problem,0.22380,-0.00745,-0.00638,0.07735,0.00501,-0.10527,0.00189,0.01796,-0.01131,0.00740
after a disappointing night at the hampton inn i moved my family here what a difference the room was spacious and comfortable the staff was suburb and super friendly \nthe only problem we had was with a chamber maid who refused to not enter the room eventhough the baby was sleeping management to their credit was very helpful and apologetic this is a place i would definitely recommend \nthis is one of theee marriott properties next to each other the others being the marriott downtown and the residence inn we were there for a conference at the marriott downtown but were able to stay at the courtyard for about less if you go out the back door the hotels are literally across the street from each other,0.17808,-0.00988,-0.00276,-0.01175,-0.01126,0.03391,0.01181,-0.02698,0.01279,-0.04538
we stayed at the marriott courtyard in downtown philadelphia for one night in august \nthe hotel is across the street from city hall and about a block from reading terminal market the hotel is also right next door to the marriott residence inn and right behind the large marriott the hotel has a landmark facade and lobby so it appears quite elegant the staff was very helpful and alert the rooms are nice but nothing super special old tvs marginal pillows \nthe indoor pool is nice for a quick cool down and the fitness center is very well equipped,0.20964,0.00112,0.00769,0.02524,0.01750,-0.09561,0.02162,-0.01594,-0.08706,0.01227
the hotels exterior blends in well with the awe inspiring city hall right across the street the interior was recently don t know when upgraded and has that newer hotel feel our room was clean and comfortable and looked like a basic courtyard hotel room we ate sandwiches at the hotels restaraunt wich was good but not great the atmosphere was relaxing and inviting wich was a needed retreat after beeing on the busy streets of philly best tips don t use mapquest to find this hotel it s not correct use the valet parking it s a steep a day but finding good parking can be difficult you will have to walk alot or take the philly flash to get around,0.18179,-0.00044,0.00266,-0.00156,0.00953,-0.07072,0.01080,-0.00178,-0.06603,0.03643
...,...,...,...,...,...,...,...,...,...,...
definitivamente recomendaría este hotel los cuartos son amplios limpios bien decorados comodos el desayuno gratis es bueno cereal leche cafe jugo fruta waffles huevo bagels mas que suficiente para empezar el día la ubicación mmmm esta un poco lejos de downtown a unos mins aproximadamente en coche \nmuy cerca del hotel hay un restaurante japones que se llama sushizushi buenisimo \nen general nuestra experiencia fue buena definitivamente regresaremos a homewood suites \nmi unica

In [155]:
words_lsa = tfidf.get_feature_names()
t_lsa = lsa.components_.argsort(axis=1)[:,-1:-7:-1]
topic_words_lsa = [[words_lsa[e] for e in l] for l in t_lsa]
topic_words_lsa

[['hotel', 'room', 'great', 'stay', 'staff', 'good'],
 ['de', 'la', 'et', 'le', 'un', 'est'],
 ['und', 'das', 'die', 'ist', 'der', 'sehr'],
 ['great', 'location', 'staff', 'friendly', 'clean', 'helpful'],
 ['di', 'la', 'el', 'con', 'una', 'que'],
 ['great', 'us', 'stay', 'service', 'staff', 'wonderful'],
 ['el', 'que', 'de', 'en', 'muy', 'es'],
 ['square', 'times', 'york', 'new', 'nyc', 'hotel'],
 ['room', 'great', 'bed', 'clean', 'bathroom', 'location'],
 ['great', 'walk', 'view', 'place', 'restaurants', 'room']]